In [ ]:
!pip list

In [ ]:
# Install required libraries
!pip install tensorflow keras matplotlib pillow numpy

In [ ]:

from google.colab import files
import os
import shutil
import matplotlib.pyplot as plt
from PIL import Image
# Import ImageDataGenerator from tensorflow.keras.preprocessing.image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import tensorflow as tf

In [ ]:
# Step 1: Install Kaggle API and set up credentials
!pip install kaggle --upgrade

# Upload your kaggle.json
from google.colab import files
files.upload()  # Upload kaggle.json here

# Set up Kaggle API credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json



In [ ]:
# Step 2: Download the dataset from Kaggle
!kaggle datasets download -d rashikrahmanpritom/plant-disease-recognition-dataset

# Step 3: Extract the downloaded dataset
import zipfile
import os

dataset_zip = 'plant-disease-recognition-dataset.zip'
dataset_dir = '/content/Dataset'

with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall(dataset_dir)

print(f"Dataset extracted to {dataset_dir}")

# Check dataset structure
!ls {dataset_dir}

In [ ]:
# Updated dataset paths
train_path = os.path.join(dataset_dir, "Train/Train")
validation_path = os.path.join(dataset_dir, "Validation/Validation")
test_path = os.path.join(dataset_dir, "Test/Test")

# Function to count files
def total_files(folder_path):
    return len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

# Display dataset stats
print("Training dataset statistics:")
for label in ["Healthy", "Powdery", "Rust"]:
    print(f" - {label}: {total_files(os.path.join(train_path, label))}")

print("\nValidation dataset statistics:")
for label in ["Healthy", "Powdery", "Rust"]:
    print(f" - {label}: {total_files(os.path.join(validation_path, label))}")

print("\nTest dataset statistics:")
for label in ["Healthy", "Powdery", "Rust"]:
    print(f" - {label}: {total_files(os.path.join(test_path, label))}")



In [ ]:
# Data augmentation and preprocessing
# Instead of importing from keras.preprocessing.image, import from tensorflow.keras.preprocessing.image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1.0 / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(train_path, target_size=(225, 225), batch_size=32, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(validation_path, target_size=(225, 225), batch_size=32, class_mode='categorical')

In [ ]:
# Model creation
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(225, 225, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

In [ ]:
# Model compilation
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(train_generator, epochs=5, validation_data=validation_generator)

In [ ]:
# Plot training history
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Save the model
model.save('leaf_disease_model.keras')

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from PIL import Image

# Function to preprocess an image
def preprocess_image(image_path, target_size=(225, 225)):
    img = load_img(image_path, target_size=target_size)
    x = img_to_array(img)
    x = x.astype('float32') / 255.0
    x = np.expand_dims(x, axis=0)
    return x

# Allow user to upload an image
print("Please upload an image for prediction:")
uploaded = files.upload()

# Check if an image is uploaded
if uploaded:
    # Get the uploaded image file name
    uploaded_image_path = list(uploaded.keys())[0]

    # Display the uploaded image
    uploaded_image = Image.open(uploaded_image_path)
    plt.imshow(uploaded_image)
    plt.axis('off')
    plt.title("Uploaded Image")
    plt.show()

    # Preprocess the uploaded image
    x = preprocess_image(uploaded_image_path)

    # Predict using the trained model
    predictions = model.predict(x)

    # Decode the prediction
    labels = {v: k for k, v in train_generator.class_indices.items()}
    predicted_label = labels[np.argmax(predictions)]
    print(f"Predicted label: {predicted_label}")

    # Display the prediction result
    print(f"The uploaded image is predicted to be: {predicted_label}")
